# Análise com SQL Avançado
## U.S. EPA Food Commodity Intake Database (FCID)
### [https://fcid.foodrisk.org/](https://fcid.foodrisk.org/)

Ativando uma conexão de banco de dados em memória usando o SGBD H2:

In [1]:
%defaultDatasource jdbc:h2:mem:db

# Importando Tabelas do FCID

In [2]:
DROP TABLE IF EXISTS Crop_Group;
DROP TABLE IF EXISTS FCID_Description;
DROP TABLE IF EXISTS Recipes;
DROP TABLE IF EXISTS Intake;

CREATE TABLE Crop_Group (
  CGN VARCHAR(2),
  CGL VARCHAR(6),
  Crop_Group_Description VARCHAR(80),
  PRIMARY KEY (CGL)
) AS SELECT
  CGN, CGL, Crop_Group_Description
FROM CSVREAD('../../data/food-intake/basics/FCID_Cropgroup_Description.csv');

CREATE TABLE FCID_Description (
  CGN VARCHAR(2),
  CG_Subgroup VARCHAR(6),
  FCID_Code VARCHAR(10),
  FCID_Desc VARCHAR(55),
  PRIMARY KEY (FCID_Code),
) AS SELECT
  cgn, CG_Subgroup, FCID_Code, FCID_Desc
FROM CSVREAD('../../data/food-intake/basics/FCID_Code_Description.csv');

CREATE TABLE Recipes (
  Food_Code VARCHAR(8),
  Mod_Code VARCHAR(8),
  Ingredient_Num TINYINT,
  FCID_Code VARCHAR(10),
  Cooked_Status TINYINT,
  Food_Form TINYINT,
  Cooking_Method TINYINT,
  Commodity_Weight DECIMAL(5, 2),
  CSFII_9498_IND TINYINT,
  WWEIA_9904_IND TINYINT,
  WWEIA_0510_IND TINYINT,
  PRIMARY KEY(Food_Code, Mod_Code, Ingredient_Num),
  FOREIGN KEY(FCID_Code)
    REFERENCES FCID_Description(FCID_Code)
      ON DELETE NO ACTION
      ON UPDATE NO ACTION
) AS SELECT
  Food_Code, Mod_Code, Ingredient_Num, FCID_Code, Cooked_Status, Food_Form, Cooking_Method,
  Commodity_Weight, CSFII_9498_IND, WWEIA_9904_IND, WWEIA_0510_IND
FROM CSVREAD('../../data/food-intake/recipes/Recipes_WWEIA_FCID_0510.csv');

CREATE TABLE Intake (
  SeqN INTEGER NOT NULL,
  DayCode TINYINT NOT NULL,
  DraBF TINYINT,
  FCID_Code VARCHAR(10),
  Cooked_Status TINYINT,
  Food_Form TINYINT,
  Cooking_Method TINYINT,
  Intake DECIMAL(13,7),
  Intake_BW DECIMAL(13,10),
  PRIMARY KEY(SeqN, DayCode, FCID_Code, Cooked_Status, Food_Form, Cooking_Method),
  FOREIGN KEY(FCID_Code)
    REFERENCES FCID_Description(FCID_Code)
      ON DELETE NO ACTION
      ON UPDATE NO ACTION
) AS SELECT
  SEQN, DAYCODE, DRABF, FCID_Code, Cooked_Status, Food_Form, Cooking_Method, Intake,Intake_BW
FROM CSVREAD('../../data/food-intake/consumption/Commodity_CSFFM_Intake_0510-cropped.csv');

# Visualizando as Tabelas

In [15]:
SELECT * FROM Crop_Group LIMIT 10;

In [4]:
SELECT * FROM FCID_Description LIMIT 10;

In [5]:
SELECT * FROM Recipes LIMIT 10;

In [7]:
SELECT * FROM Intake LIMIT 10;

# Métricas

Considere que a tabela Intake registra alimentos consumidos por 1.489 pessoas. Considere as seguintes métricas para um alimento:

| Métrica | Descrição |
| --- | --- |
| Popularidade | número de pessoas (dentre as 1.489) que consumiram o alimento |
| Intake_Sum | total consumido do alimento pelas 1.489 pessoas em gramas |
| Intake_AVG | média de consumo do alimento em gramas |
| Intake_AVG_BW | média de consumo do alimento x peso da pessoa |
| Recipes | número de receitas (dentre as 7.154 receitas) que têm o produto como ingrediente |

## 1)  Construa uma View que apresente essas métricas por produto

* Veja exemplo em: `/data/food-intake/computed/commodity-profile.csv`
* Importante: esta tabela foi feita com um número maior de registros, portanto os valores não serão iguais aos seus

In [ ]:
CREATE VIEW INTAKE_V1 AS
SELECT I.FCID_Code, 
COUNT(DISTINCT(I.SEQN))AS POPULARITY, 
SUM(I.INTAKE) AS INTAKE_SUM, 
AVG(I.INTAKE_BW) AS INTAKE_AVG_BW
FROM Intake I
GROUP BY I.FCID_Code
ORDER BY I.FCID_Code;

CREATE VIEW INTAKE_V2 AS
SELECT I.FCID_Code, AVG(I.INTAKE) AS INTAKE_AVG
FROM Intake I
GROUP BY I.FCID_Code;

CREATE VIEW RECIPES_V1 AS
SELECT R.FCID_Code, COUNT(DISTINCT(R.FOOD_CODE)) AS RECIPES
FROM Recipes R
GROUP BY R.FCID_Code;

CREATE VIEW FOOD_ANALYSIS AS
SELECT FD.FCID_Code, FD. FCID_Desc, FD.CGN, FD.CG_SUBGROUP, IV1.POPULARITY, IV1.INTAKE_SUM, IV2.INTAKE_AVG, IV1.INTAKE_AVG_BW, RV1.RECIPES
FROM FCID_Description FD, INTAKE_V1 IV1, INTAKE_V2 IV2, RECIPES_V1 RV1
WHERE IV1.FCID_Code = FD.FCID_Code
AND IV2.FCID_Code = FD.FCID_Code
AND RV1.FCID_Code = FD.FCID_Code;

## 2) Como você analisaria a correlação entre as métricas?

* Por exemplo, produtos mais populares são mais consumidos (em número de pessoas ou em quantidade)?
* Proponha uma ou mais queries para fazer esta análise

Propostas:

* Produtos mais populares x Mais consumidos
* Valor médio do produto por receita x Consumo médio

In [ ]:
-- Mais consumidos são os mais populares?
SELECT FA.FCID_CODE, FA.FCID_DESC, FA.INTAKE_SUM, FA.POPULARITY, CAST(FA.POPULARITY AS FLOAT)/(
	SELECT COUNT(DISTINCT(I.SEQN))
	FROM INTAKE I
) AS POPULARITY_PERCENTAGE
FROM FOOD_ANALYSIS FA
ORDER BY FA.INTAKE_SUM DESC
LIMIT 15;

In [ ]:
-- Produtos que estão em mais receitas são os mais populares?
SELECT FA.FCID_CODE, FA.FCID_DESC, FA.RECIPES, FA.POPULARITY, CAST(FA.POPULARITY AS FLOAT)/(
	SELECT COUNT(DISTINCT(I.SEQN))
	FROM INTAKE I
) AS POPULARITY_PERCENTAGE
FROM FOOD_ANALYSIS FA
ORDER BY FA.RECIPES DESC
LIMIT 15

## 3) Podemos criar grupos de consumidores conforme um perfil?
* por exemplo, consumidores podem ser agrupados por alimentos que comem predominantemente?
* como você associaria grupos a classes?

Propostas:

* Consumidores por alimentos que não consomem (onívoros, vegetarianos, veganos)

In [ ]:
CREATE VIEW ONIVOROS AS
SELECT DISTINCT(I.SEQN)
FROM INTAKE I
JOIN FCID_Description D ON D.FCID_CODE = I.FCID_CODE
WHERE ((D.CGN >= 31 AND D.CGN <= 80) AND D.CGN NOT IN (36,37,70));

CREATE VIEW VEGETARIANOS AS
SELECT DISTINCT(I.SEQN)
FROM INTAKE I
WHERE I.SEQN NOT IN (
    SELECT *
    FROM ONIVOROS
);

CREATE VIEW VEGANOS AS
SELECT DISTINCT(I.SEQN)
FROM INTAKE I
WHERE I.SEQN NOT IN (
    SELECT DISTINCT(I2.SEQN)
    FROM INTAKE I2
    JOIN FCID_Description D2 ON D2.FCID_CODE = I2.FCID_CODE
    WHERE (D2.CGN >= 31 AND D2.CGN <= 80)
)

In [ ]:
--Número de pessoas onívaras
SELECT COUNT(DISTINCT(SEQN))
FROM ONIVOROS

In [ ]:
--Número de pessoas vegetarianas
SELECT COUNT(DISTINCT(SEQN))
FROM VEGETARIANOS

In [ ]:
--Número de pessoas veganas
SELECT COUNT(DISTINCT(SEQN))
FROM VEGANOS

## 4) Que métricas podem ser analisadas para a comparação de perfis?
* escreva uma query SQL que calcule pelo menos uma métrica comparativa

Propostas:

* Consumo médio de um alimento (dentro do grupo)
* Peso médio das pessoas em um grupo (acho que não dá)
* Número de refeições médias por grupo
* Variação do consumo intra-grupo (maior consumo menos menor consumo)

In [ ]:
-- Comidas mais populares entre os veganos
SELECT DISTINCT(FA.FCID_Code), FA. FCID_Desc, FA.POPULARITY
FROM FOOD_ANALYSIS FA
JOIN INTAKE I ON I.FCID_CODE = FA.FCID_CODE
WHERE I.SEQN IN (
    SELECT *
    FROM VEGANOS
)
ORDER BY FA.POPULARITY DESC
LIMIT 10

In [ ]:
-- Comidas mais populares entre os vegetarianos
SELECT DISTINCT(FA.FCID_Code), FA. FCID_Desc, FA.POPULARITY
FROM FOOD_ANALYSIS FA
JOIN INTAKE I ON I.FCID_CODE = FA.FCID_CODE
WHERE I.SEQN IN (
    SELECT *
    FROM VEGETARIANOS
)
ORDER BY FA.POPULARITY DESC
LIMIT 10

In [ ]:
-- Comidas mais populares entre os onívoros
SELECT DISTINCT(FA.FCID_Code), FA. FCID_Desc, FA.POPULARITY
FROM FOOD_ANALYSIS FA
JOIN INTAKE I ON I.FCID_CODE = FA.FCID_CODE
WHERE I.SEQN IN (
    SELECT *
    FROM ONIVOROS
)
ORDER BY FA.POPULARITY DESC
LIMIT 10